In [8]:
from tkinter import *
from tkinter import filedialog
import numpy as np
import random
import pickle

# 黑白棋盤初始化 0:空、1:白、2:黑、3:可下點
board = np.array([[0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0],
                  [0,0,0,3,0,0,0,0],
                  [0,0,3,1,2,0,0,0],
                  [0,0,0,2,1,3,0,0],
                  [0,0,0,0,3,0,0,0],
                  [0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0]])
initBoard = board.copy()
# 盤面權重
weight = np.array([[9,-3,5,5,5,5,-3,9],
                   [-3,-3,6,6,6,6,-3,-3],
                   [5,6,6,6,6,6,6,5],
                   [5,6,6,9,9,6,6,5],
                   [5,6,6,9,9,6,6,5],
                   [5,6,6,6,6,6,6,5],
                   [-3,-3,6,6,6,6,-3,-3],
                   [9,-3,5,5,5,5,-3,9]])
initweight = weight.copy()
# 紀錄所有棋步 & 換手 & 位置 & 筆記
boardRecord = [board.copy()]
whoRecord = [2]
nowPos = 0
notelist = [''] * 64
gameover = 0
# 0:關閉、1:白、2:黑
aiturn = 0


# 掃描 board 畫出棋子
def render():
    global board, notelist, nowPos
    for i in range(8):
        for j in range(8):
            if board[i][j] == 0:
                gameBox.create_oval(i*60+5, j*60+5, i*60+55, j*60+55, fill='#00cc00', outline='#00cc00')
            elif board[i][j] == 1:
                gameBox.create_oval(i*60+5, j*60+5, i*60+55, j*60+55, fill='white', outline='white')
            elif board[i][j] == 2:
                gameBox.create_oval(i*60+5, j*60+5, i*60+55, j*60+55, fill='black', outline='black')
            elif board[i][j] == 3:
                gameBox.create_oval(i*60+5, j*60+5, i*60+55, j*60+55, fill='#00cc00', outline='#00cc00')
                gameBox.create_oval(i*60+25, j*60+25, i*60+35, j*60+35, fill='yellow', outline='yellow')
    # 比起update()，update_idletasks不會直接Block住正在執行的東西，會等idle的時間運行
    gameBox.update_idletasks()
    # 筆記
    note.delete('1.0', END)
    note.insert(END, notelist[nowPos])
    # 第幾手
    pos['text'] = str(nowPos)
                
# 偵測滑鼠在棋盤的位置
def mouseCatch(event):
    global board, whoRecord, boardRecord, nowPos, aiturn, whoturn, final_x, final_y, gameover, weight
    mx = int(event.x/60)
    my = int(event.y/60)
    if aiturn != whoturn:
        if board[mx][my] == 3:
            # 若下了新將會洗掉原本的Record
            if nowPos < (len(whoRecord)-1):
                del boardRecord[nowPos+1:]
                del whoRecord[nowPos+1:]
            nowboard = board.copy()
            whoturn, chesscount, gameover = reversi(nowboard, mx, my, whoturn)
            # 根據下棋點調整權重矩陣 1代表人
            weightChange(final_x, final_y, 1)
            nowPos += 1
            #紀錄運算結果
            board = nowboard.copy()
            # 紀錄
            boardRecord.append(board)
            whoRecord.append(whoturn)
            # 計算過的結果 => 渲染
            render()
            numHint()
    if aiturn == whoturn:
        # 檢查是否輪到AI
        # 是否換AI下棋、只有在最新的一手才會動作
        if aiturn != 0 and nowPos+1 == len(whoRecord) and aiturn == whoturn:
            while whoturn == aiturn and gameover == 0:
                #print('#============================#')
                numHint()
                runAI()
                nowboard = board.copy()
                whoturn, chesscount, gameover = reversi(nowboard, final_x, final_y, whoturn)
                #print(whoturn, chesscount, gameover, final_x, final_y)
                # 根據下棋點調整權重矩陣 0代表AI
                weightChange(final_x, final_y, 0)
                nowPos += 1
                # 紀錄運算結果
                board = nowboard.copy()
                # 紀錄
                boardRecord.append(board)
                whoRecord.append(whoturn)
                # 計算過的結果 => 渲染
                render()
                numHint()

# 更改權重
def weightChange(x, y, who):
    global weight
    # AI下到角落則調整相鄰兩排邊的權重
    if x == 0 and y == 0:
        weight[x+1][y+1] = 9
        for i in range(1,7):
            weight[x+i][y] = 10-i
            weight[x][y+i] = 10-i
    elif x == 7 and y == 0:
        weight[x-1][y+1] = 9
        for i in range(1,7):
            weight[x-i][y] = 10-i
            weight[x][y+i] = 10-i
    elif x == 7 and y == 7:
        weight[x-1][y-1] = 9
        for i in range(1,7):
            weight[x-i][y] = 10-i
            weight[x][y-i] = 10-i
    elif x == 0 and y == 7:
        weight[x+1][y-1] = 9
        for i in range(1,7):
            weight[x+i][y] = 10-i
            weight[x][y-i] = 10-i
        
# AI
tmp_x, tmp_y, final_x, final_y, tmp_chesscount, final_min = -1, -1, -1, -1, -1, 100
def myAI(whoturn, board, deep, aiturn, total, weight):
    global final_min, final_x, final_y, tmp_x, tmp_y, tmp_chesscount
    # 開局前30手
    # 向下遞迴尋找對方可下點、自己翻棋數目
    howdeep = 3
    if total < 20:
        howdeep = 1
    elif total >= 20 and total < 40:
        howdeep = 3
    elif total >= 40:
        howdeep = 1
    deep += 1
    for i in range(8):
        for j in range(8):
            if board[i][j] == 3:
                aiboard = board.copy()
                whonext, chesscount, g = reversi(aiboard, i, j, whoturn)
                #print(chesscount)
                #print(aiboard)
                # 進入第一層迴圈
                if deep == 1:
                    tmp_x = i
                    tmp_y = j
                    tmp_chesscount = chesscount

                # 進入最後一層遞迴
                if deep == howdeep:
                    count = 0
                    for q in range(8):
                        for p in range(8):
                            if aiboard[q][p] == 3:
                                count += 1
                    # AI決定機制
                    if total > 50:
                        decision = tmp_chesscount-weight[tmp_x][tmp_y]*2
                    else:
                        decision = count+((tmp_chesscount*3)/int(total**0.5))-weight[tmp_x][tmp_y]*3
                    if decision == final_min:
                        random_point = bool(random.getrandbits(1))
                        if random_point == True:
                            final_min = decision
                            final_x = tmp_x
                            final_y = tmp_y
                    elif decision < final_min:
                        final_min = decision
                        final_x = tmp_x
                        final_y = tmp_y
                    #print('剛剛AI翻了多少棋子:', tmp_chesscount, '對方未來的可下點:', count, 'decision:', decision, '下棋點:', tmp_x, tmp_y, gameover)
                # 進入下一層遞迴預測
                if deep < howdeep:
                    aiboard_d = aiboard.copy()
                    myAI(whonext, aiboard_d, deep, aiturn, total, weight)
    
# 盤面規則計算
def reversi(nowboard, mx, my, whoturn):
    global board, boardRecord, whoRecord
    gameover = 0
    # 黑棋下子並換手
    if whoturn == 2:
        nowboard[mx][my] = 2
        chesscount = turnOver(nowboard, mx, my, 2)
        whoturn = 1
    # 白旗下子並換手
    else:
        nowboard[mx][my] = 1
        chesscount = turnOver(nowboard, mx, my, 1)
        whoturn = 2
    # 計算有無可下點，無則換手重新計算可下點，若再無則gameover
    permit = 0
    for i in range(8):
        for j in range(8):
            if nowboard[i][j] == 3:
                nowboard[i][j] = 0
            if nowboard[i][j] == 0:
                tryboard = nowboard.copy()
                turnOver(tryboard, i, j, whoturn)
                if (tryboard != nowboard).any():
                    permit = 1
                    nowboard[i][j] = 3
    if permit == 0:
        if whoturn == 2:
            whoturn = 1
        else:
            whoturn = 2
        for i in range(8):
            for j in range(8):
                if nowboard[i][j] == 3:
                    nowboard[i][j] = 0
                if nowboard[i][j] == 0:
                    tryboard = nowboard.copy()
                    turnOver(tryboard, i, j, whoturn)
                    if (tryboard != nowboard).any():
                        permit = 1
                        nowboard[i][j] = 3
    if permit == 0 :
        gameover = 1
        
    # print(chesscount)
    return whoturn, chesscount, gameover
    
total = 4
# 顯示盤面棋數 & 下棋方提示
def numHint():
    global total, whoturn, board
    whitenum = 0
    blacknum = 0
    for i in range(8):
        for j in range(8):
            if board[i][j] == 1:
                whitenum += 1
            elif board[i][j] == 2:
                blacknum += 1
    total = whitenum + blacknum
    wnum['text'] = str(whitenum)
    bnum['text'] = str(blacknum)
    if whoturn == 1:
        wnum['relief'] = "solid"
        bnum['relief'] = "flat"
    elif whoturn == 2:
        bnum['relief'] = "solid"
        wnum['relief'] = "flat"
            

# 帶入新點 => 依照黑白棋規則翻面
def turnOver(nowboard, mx, my, whoturn):
    # 記錄翻了多少棋子
    chesscount = 0
    # 往上尋找
    for i in range(my-1, -1, -1):
        if nowboard[mx][i] == 0 or nowboard[mx][i] == 3:
            break
        if nowboard[mx][i] == whoturn:
            for j in range(my-1, i, -1):
                nowboard[mx][j] = whoturn
                chesscount += 1
            break
    # 往右尋找
    for i in range(mx+1, 8):
        if nowboard[i][my] == 0 or nowboard[i][my] == 3:
            break
        if nowboard[i][my] == whoturn:
            for j in range(mx+1, i):
                nowboard[j][my] = whoturn
                chesscount += 1
            break
    # 往下尋找
    for i in range(my+1, 8):
        if nowboard[mx][i] == 0 or nowboard[mx][i] == 3:
            break
        if nowboard[mx][i] == whoturn:
            for j in range(my+1, i):
                nowboard[mx][j] = whoturn
                chesscount += 1
            break
    # 往左尋找
    for i in range(mx-1, -1, -1):
        if nowboard[i][my] == 0 or nowboard[i][my] == 3:
            break
        if nowboard[i][my] == whoturn:
            for j in range(mx-1, i, -1):
                nowboard[j][my] = whoturn
                chesscount += 1
            break
    # 往右上尋找
    for i in range(1, 8):
        if (mx+i) <= 7 and (my-i) >= 0:
            if nowboard[mx+i][my-i] == 0 or nowboard[mx+i][my-i] == 3:
                break
            if nowboard[mx+i][my-i] == whoturn:
                for j in range(1, i):
                    nowboard[mx+j][my-j] = whoturn
                    chesscount += 1
                break
    # 往右下尋找
    for i in range(1, 8):
        if (mx+i) <= 7 and (my+i) <= 7:
            if nowboard[mx+i][my+i] == 0 or nowboard[mx+i][my+i] == 3:
                break
            if nowboard[mx+i][my+i] == whoturn:
                for j in range(1, i):
                    nowboard[mx+j][my+j] = whoturn
                    chesscount += 1
                break
    # 往左下尋找
    for i in range(1, 8):
        if (mx-i) >= 0 and (my+i) <= 7:
            if nowboard[mx-i][my+i] == 0 or nowboard[mx-i][my+i] == 3:
                break
            if nowboard[mx-i][my+i] == whoturn:
                for j in range(1, i):
                    nowboard[mx-j][my+j] = whoturn
                    chesscount += 1
                break
    # 往左上尋找
    for i in range(1, 8):
        if (mx-i) >= 0 and (my-i) >= 0:
            if nowboard[mx-i][my-i] == 0 or nowboard[mx-i][my-i] == 3:
                break
            if nowboard[mx-i][my-i] == whoturn:
                for j in range(1, i):
                    nowboard[mx-j][my-j] = whoturn
                    chesscount += 1
                break
    
    return chesscount

# 回顧功能   
def recordView(way):
    global whoRecord, boardRecord, nowPos, board, whoturn
    if way == 0:
        if (nowPos-1) >= 0:
            board = boardRecord[nowPos-1].copy()
            whoturn = whoRecord[nowPos-1]
            nowPos -= 1
    elif way == 1:
        if (nowPos+1) < (len(boardRecord)):
            board = boardRecord[nowPos+1].copy()
            whoturn = whoRecord[nowPos+1]
            nowPos += 1
    elif way == 2:
        board = boardRecord[0].copy()
        whoturn = whoRecord[0]
        nowPos = 0
    elif way == 3:
        board = boardRecord[-1].copy()
        whoturn = whoRecord[-1]
        nowPos = len(boardRecord)-1
    render()
    numHint()

def runAI():
    global tmp_x, tmp_y, final_x, final_y, tmp_chesscount, final_min, board, whoturn, aiturn, total, weight
    tmp_x, tmp_y, final_x, final_y, tmp_chesscount, final_min = -1, -1, -1, -1, -1, 100
    myAI(whoturn, board, 0, aiturn, total, weight)
# 開新的一局
def restart():
    global board, boardRecord, whoRecord, initBoard, whoturn, nowPos, aiturn, total, gameover, weight, initweight, final_x, final_y
    total = 0
    gameover = 0
    notelist = [''] * 64
    board = initBoard.copy()
    weight = initweight.copy()
    whoturn = 2
    del boardRecord[1:]
    del whoRecord[1:]
    nowPos = 0
    render()
    numHint()
    if aiturn != 0:
        aiturn = whoturn
        runAI()
        nowboard = board.copy()
        whoturn, chesscount, g = reversi(nowboard, final_x, final_y, whoturn)
        nowPos += 1
        board = nowboard.copy()
        # 計算過的結果 => 渲染
        render()
        numHint()
        # 紀錄
        boardRecord.append(board)
        whoRecord.append(whoturn)
    
# 模式切換
def switchMode(mode):
    global whoturn, aiturn, board, nowPos, total, final_x, final_y
    # 切換至player mode
    if mode == 0:
        player['relief'] = SUNKEN
        player['state'] = DISABLED
        ai['relief'] = RAISED
        ai['state'] = NORMAL
        aiturn = 0
    # 切換至AI mode
    else:
        ai['relief'] = SUNKEN
        ai['state'] = DISABLED
        player['relief'] = RAISED
        player['state'] = NORMAL
        aiturn = whoturn
        runAI()
        nowboard = board.copy()
        whoturn, chesscount, g = reversi(nowboard, final_x, final_y, whoturn)
        nowPos += 1
        board = nowboard.copy()
        # 計算過的結果 => 渲染
        render()
        numHint()
        # 紀錄
        boardRecord.append(board)
        whoRecord.append(whoturn)
        
def saveData():
    global boardRecord, whoRecord, nowPos, notelist
    picklelist = [boardRecord, whoRecord, nowPos, notelist]
    root.filename =  filedialog.asksaveasfilename(initialdir = ".")
    try:
        file = open(root.filename, 'wb')
        pickle.dump(picklelist, file)
        file.close()
    except:
        pass
        
def openData():
    global boardRecord, whoRecord, nowPos, board, whoturn, notelist
    root.filename =  filedialog.askopenfilename(initialdir = ".")
    try:
        file = open(root.filename, "rb")
        picklelist = pickle.load(file, encoding='iso-8859-1')
        file.close()
        boardRecord = picklelist[0]
        whoRecord = picklelist[1]
        nowPos = picklelist[2]
        notelist = picklelist[3]
        board = boardRecord[nowPos].copy()
        whoturn = whoRecord[nowPos]
        render()
        numHint()
    except:
        pass
    
def getnote(event):
    global nowPos, notelist
    mynote = note.get("1.0","end-1c")
    notelist[nowPos] = mynote

#=================================================#
#               My Revesi Project                 #
#=================================================#

root =Tk()
root.state("zoomed")

# 棋盤畫線
myframe = Frame(root, bg='black', padx=2, pady=2)
myframe.place(x=50, y=50)
mysize = 480
gameBox = Canvas(myframe, width=mysize, height=mysize, bg='#00cc00')
gameBox.pack()
for i in range(1,8):
    gameBox.create_line(60*i, 0, 60*i, mysize+2, width=2, fill='white')
    gameBox.create_line(0, 60*i, mysize+2, 60*i, width=2, fill='white')

# 功能按鈕
restart = Button(root, text="Restart", width=12, height=1, bg='#55ee55', font=('Arial', 20), command=restart)
restart.place(x=600, y=50)
player = Button(root, text="Player Mode", relief=SUNKEN, state=DISABLED, width=12, height=1, bg='#55ee55', font=('Arial', 20), command=lambda:switchMode(0))
player.place(x=600, y=125)
ai = Button(root, text="AI Mode",  width=12, height=1, bg='#55ee55', font=('Arial', 20), command=lambda:switchMode(1))
ai.place(x=600, y=200)
old = Button(root, text="Open", width=12, height=1, bg='#55ee55', font=('Arial', 20), command=openData)
old.place(x=600, y=275)
save = Button(root, text="Save", width=12, height=1, bg='#55ee55', font=('Arial', 20), command=saveData)
save.place(x=600, y=350)
left = Button(root, text="◀", width=3, height=0, bg='#55ee55', font=('Arial', 16), command=lambda:recordView(0))
left.place(x=652, y=490)
right = Button(root, text="▶", width=3, height=0, bg='#55ee55', font=('Arial', 16), command=lambda:recordView(1))
right.place(x=704, y=490)
leftall = Button(root, text="◁", width=3, height=0, bg='#55ee55', font=('Arial', 16), command=lambda:recordView(2))
leftall.place(x=600, y=490)
rightall = Button(root, text="▷", width=3, height=0, bg='#55ee55', font=('Arial', 16), command=lambda:recordView(3))
rightall.place(x=756, y=490)
notelabel = Label(root, text="Notebook", font=('Arial', 20, 'bold'))
notelabel.place(x=850, y=40)
note = Text(root, bg="#bbffbb", width=25, height=15, font=('Arial', 20))
note.place(x=850, y=80)
note.bind("<Leave>", getnote)
pos = Label(root, text='0', font=('Arial', 20))
pos.place(x=680, y=450)

# 數字顯示
wnum = Label(root, text="2", width=6, borderwidth=3, bg='#00cc00', fg='white', font=('Arial', 50))
wnum.place(x=50, y=550)
bnum = Label(root, text="2", width=6, borderwidth=3, bg='#00cc00', fg='black', font=('Arial', 50))
bnum.place(x=295, y=550)

# 初始化棋盤
render()
# 黑棋先手 黑:2、白:1
whoturn = 2
numHint()
# 滑鼠偵測
gameBox.bind("<Button-1>", mouseCatch)

root.mainloop()